In [1]:
!sudo apt-get install tesseract-ocr
# !pip install torchvision==0.13.1
!pip install opencv-python
!pip install matplotlib
!pip install numpy
!pip install pytesseract
# !pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1
!pip install opencv-python-headless

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,967 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [2]:
!git clone https://github.com/clovaai/CRAFT-pytorch
%cd CRAFT-pytorch

import gdown

# 파일 ID를 사용하여 다운로드 링크 생성
file_id = '1Jk4eGD7crsqCCg9C9VjCLkMN3ze8kutZ'
url = f'https://drive.google.com/uc?id={file_id}'

# 파일 다운로드
output = 'craft_mlt_25k.pth'
gdown.download(url, output, quiet=False)

Cloning into 'CRAFT-pytorch'...
remote: Enumerating objects: 59, done.
remote: Total 59 (delta 0), reused 0 (delta 0), pack-reused 59 (from 1)
Receiving objects: 100% (59/59), 1.69 MiB | 15.74 MiB/s, done.
Resolving deltas: 100% (25/25), done.
/content/CRAFT-pytorch


Downloading...
From: https://drive.google.com/uc?id=1Jk4eGD7crsqCCg9C9VjCLkMN3ze8kutZ
To: /content/CRAFT-pytorch/craft_mlt_25k.pth
100%|██████████| 83.2M/83.2M [00:00<00:00, 89.3MB/s]


'craft_mlt_25k.pth'

In [3]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from craft import CRAFT
from craft_utils import getDetBoxes, adjustResultCoordinates
from imgproc import loadImage, normalizeMeanVariance, resize_aspect_ratio
import pytesseract

In [14]:
# Colab에 Google Drive 연동
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 4. 모델 로드 함수

def load_model():
    model = CRAFT()
    model.load_state_dict(torch.load('craft_mlt_25k.pth', map_location='cuda' if torch.cuda.is_available() else 'cpu'), strict=False)
    model.eval()
    return model
# def load_model():
#     model = CRAFT()  # CRAFT 모델 인스턴스 생성
#     model.load_state_dict(torch.load('craft_mlt_25k.pth', map_location='cpu'))
#     model.eval()  # 평가 모드로 설정
#     return model

# 5. 이미지 전처리 함수
# def preprocess_image(image_path):
#     image = loadImage(image_path)
#     img_resized, target_ratio, size_heatmap = resize_aspect_ratio(image, 1280, interpolation=cv2.INTER_LINEAR, mag_ratio=1.5)
#     img_normalized = normalizeMeanVariance(img_resized)
#     img_normalized = torch.from_numpy(img_normalized).permute(2, 0, 1).unsqueeze(0)
#     return img_normalized, size_heatmap, target_ratio

# def preprocess_image(image_path):
#     image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
#     _, image_bin = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#     image_blurred = cv2.GaussianBlur(image_bin, (5, 5), 0)
#     img_resized, target_ratio, size_heatmap = resize_aspect_ratio(image_blurred, 1280, interpolation=cv2.INTER_LINEAR, mag_ratio=1.5)
#     img_normalized = normalizeMeanVariance(img_resized)
#     img_normalized = torch.from_numpy(img_normalized).permute(2, 0, 1).unsqueeze(0)
#     return img_normalized, size_heatmap, target_ratio

def preprocess_image(image_path):
    # 이미지를 그레이스케일로 읽기
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # 이진화 적용
    _, image_bin = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # 가우시안 블러 적용
    image_blurred = cv2.GaussianBlur(image_bin, (5, 5), 0)
    # 이미지 크기 조정
    img_resized, target_ratio, size_heatmap = resize_aspect_ratio(image_blurred, 1280, interpolation=cv2.INTER_LINEAR, mag_ratio=1.5)
    # 채널을 추가하여 3채널 이미지로 변환
    if len(img_resized.shape) == 2:  # 흑백 이미지
        img_resized = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)

    img_normalized = normalizeMeanVariance(img_resized)
    img_normalized = torch.from_numpy(img_normalized).permute(2, 0, 1).unsqueeze(0)
    return img_normalized, size_heatmap, target_ratio


# 6. 텍스트 박스 추출 함수 (모델 출력 확인 추가)
# def extract_textboxes(image_path, model):
#     img_normalized, size_heatmap, target_ratio = preprocess_image(image_path)
#     with torch.no_grad():
#         y, feature = model(img_normalized)

#         # 모델 출력 확인
#         print(f"Model output (y): {type(y)}, shape: {y.shape}")
#         print(f"Model output (feature): {type(feature)}, shape: {feature.shape}")

#     # y와 feature가 예상되는 형태인지 확인하고 이후 코드 수정
#     boxes, polys = getDetBoxes(y, 0.7, 0.4, 0.4, True)
#     boxes = adjustResultCoordinates(boxes, target_ratio)
#     return boxes

# 6. 텍스트 박스 추출 함수 (adjustResultCoordinates 수정)
def extract_textboxes(image_path, model):
    img_normalized, size_heatmap, target_ratio = preprocess_image(image_path)
    with torch.no_grad():
        y, _ = model(img_normalized)

    # y 텐서에서 textmap과 linkmap 분리
    y = y.squeeze(0).cpu().numpy()  # (height, width, 2)
    textmap = y[:, :, 0]  # 텍스트 영역 맵
    linkmap = y[:, :, 1]  # 링크 영역 맵

    # # getDetBoxes를 호출할 때 textmap과 linkmap 전달
    # boxes, polys = getDetBoxes(textmap, linkmap, 0.7, 0.4, 0.4, True)
    # 예시: 임계값 조정
    boxes, polys = getDetBoxes(textmap, linkmap, text_threshold=0.6, link_threshold=0.3, low_text=0.2, poly=False)

    # adjustResultCoordinates를 호출할 때, 너비와 높이 비율 모두 전달
    boxes = adjustResultCoordinates(boxes, ratio_w=target_ratio, ratio_h=target_ratio)
    return boxes


# 7. 결과 시각화 함수
def visualize_results(image_path, boxes):
    image = cv2.imread(image_path)
    for box in boxes:
        box = np.array(box).astype(np.int32).reshape((-1, 1, 2))
        cv2.polylines(image, [box], isClosed=True, color=(0, 255, 0), thickness=2)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()


# 8. OCR로 텍스트 추출 함수
def extract_text_from_boxes(image_path, boxes):
    image = cv2.imread(image_path)
    extracted_texts = []
    for box in boxes:
        # 각 박스의 좌표를 사용해 이미지에서 영역을 잘라내기
        pts = np.array(box).reshape((-1, 2))
        rect = cv2.boundingRect(pts)
        x, y, w, h = rect
        cropped_img = image[y:y+h, x:x+w]

        # 잘라낸 이미지에서 텍스트 추출
        text = pytesseract.image_to_string(cropped_img, lang='eng')
        extracted_texts.append(text.strip())

    return extracted_texts

# imgproc.py의 resize_aspect_ratio 함수 수정
def resize_aspect_ratio(img, square_size, interpolation=cv2.INTER_LINEAR, mag_ratio=1):
    if len(img.shape) == 2:  # 흑백 이미지
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    height, width, channel = img.shape

    # 이미지 크기 조정 비율 계산
    img_w = int(width * mag_ratio)
    img_h = int(height * mag_ratio)

    # 크기 조정
    img_resized = cv2.resize(img, (img_w, img_h), interpolation=interpolation)

    # 패딩 추가
    delta_w = max(square_size - img_w, 0)
    delta_h = max(square_size - img_h, 0)

    top = delta_h // 2
    bottom = delta_h - top
    left = delta_w // 2
    right = delta_w - left

    # 패딩을 추가하여 이미지 크기 맞추기
    if img_resized.shape[2] == 3:  # 컬러 이미지
        img_padded = cv2.copyMakeBorder(img_resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(128, 128, 128))
    else:  # 흑백 이미지
        img_padded = cv2.copyMakeBorder(img_resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=128)

    return img_padded, img_w / width, img_h / height


In [15]:
# 9. 메인 실행 부분

# 테스트를 위한 이미지 파일 설정
test_png_file_path = '/content/drive/MyDrive/Colab Notebooks/DL project/data/image/P.Paper/O.Form/IMG_OCR_53_4PR_09180.png'

# 메인 실행 부분에서 OCR 함수 호출
if __name__ == '__main__':
    model = load_model()
    image_path = test_png_file_path  # 이미지 파일 경로를 지정하세요.

    # 이미지가 정상적으로 업로드되었는지 확인
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image from {image_path}. Please check the file path and permissions.")
    else:
        boxes = extract_textboxes(image_path, model)
        visualize_results(image_path, boxes)  # 시각화
        texts = extract_text_from_boxes(image_path, boxes)  # 텍스트 추출
        print("Extracted Texts:")
        for text in texts:
            print(text)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and may be removed in the future. Please access them via the appropriate Weights Enum instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Error: Could not load image from /content/drive/MyDrive/Colab Notebooks/DL project/data/image/P.Paper/O.Form/IMG_OCR_53_4PR_09180.png. Please check the file path and permissions.
